# Options for Accessing ATL24

### Imports

In [ ]:
from sliderule import sliderule
import matplotlib.pyplot as plt
import numpy as np

### Configuration

In [ ]:
# configure sliderule to output verbose log messages
sliderule.init(verbose=True)

### Plotting Helper Variables and Functions

In [ ]:
# color map for ATL24 classifications
COLORS = {
    0: ['gray', 'unclassified'],
    40: ['red', 'bathymetry'],
    41: ['blue', 'sea_surface']
}

In [ ]:
# plot ATL24 dataframes
def plot_atl24(gdf, x_min, x_max, column=None):
    start_xatc = np.min(gdf['x_atc'])
    fig,ax = plt.subplots(1, 1, figsize=(18, 6), constrained_layout=True)
    if column == None:
        for class_val, color_name in COLORS.items():
            ii=gdf["class_ph"]==class_val
            ax.plot(gdf['x_atc'][ii]-start_xatc, gdf['ortho_h'][ii], 'o', markersize=1, color=color_name[0], label=color_name[1])
    else:
        sc = ax.scatter(gdf['x_atc']-start_xatc, gdf['ortho_h'], c=gdf[column], cmap='viridis')
    ax.set_xlim(x_min, x_max)
    plt.show()

In [ ]:
# plot ATL06 dataframes
def plot_atl06(gdf, x_min, x_max, column=None):
    start_xatc = np.min(gdf['x_atc'])
    fig,ax = plt.subplots(1, 1, figsize=(18, 6), constrained_layout=True)
    ax.plot(gdf['x_atc']-start_xatc, gdf['h_mean'], 'o', markersize=5, color='red')
    ax.set_xlim(x_min, x_max)
    plt.show()

In [ ]:
# plot ATL03 dataframes
def plot_atl03(gdf, x_min, x_max, column=None):
    start_xatc = np.min(gdf['x_atc'])
    fig,ax = plt.subplots(1, 1, figsize=(18, 6), constrained_layout=True)
    if column == None:
        for class_val, color_name in COLORS.items():
            ii=gdf["atl24_class"]==class_val
            ax.plot(gdf['x_atc'][ii]-start_xatc, gdf['height'][ii], 'o', markersize=1, color=color_name[0], label=color_name[1])
    else:
        sc = ax.scatter(gdf['x_atc']-start_xatc, gdf['height'], c=gdf[column], cmap='viridis')
    ax.set_xlim(x_min, x_max)
    plt.show()

### Define Area of Interest (north short of Dominican Republic)

In [ ]:
aoi = [ { "lat": 19.42438470712139, "lon": -69.79907695695609  },
        { "lat": 19.31125534696085,  "lon": -69.79907695695609 },
        { "lat": 19.31125534696085,  "lon": -69.33527941905237 },
        { "lat": 19.42438470712139,  "lon": -69.33527941905237 },
        { "lat": 19.42438470712139,  "lon": -69.79907695695609 } ]

# (1) Quick Access

In [ ]:
gdf1 = sliderule.run("atl24x", {}, aoi=aoi)

In [ ]:
gdf1

In [ ]:
gdf1.plot(column='ortho_h', cmap='viridis')

# (2) Access a Single Track

In [ ]:
gdf2 = sliderule.run("atl24x", {"beams": "gt3r", "rgt": 202, "cycle": 12}, aoi=aoi)

In [ ]:
gdf2

In [ ]:
plot_atl24(gdf2, 0, 2500)

# (3) Detailed Access of a Single Track

In [ ]:
parms = {
    "atl24": {
        "compact": False,
        "confidence_threshold": 0.0,
        "class_ph": ["unclassified", "sea_surface", "bathymetry"]
    },
    "beams": "gt3r",
    "rgt": 202,
    "cycle": 12
}
gdf3 = sliderule.run("atl24x", parms, aoi=aoi)

In [ ]:
gdf3

In [ ]:
plot_atl24(gdf3, 0, 2500)

In [ ]:
plot_atl24(gdf3, 0, 2500, "confidence")

# (4) Access All ATL03 Photons using ATL24 as a Classifier

In [ ]:
parms = {
    "atl24": {
        "class_ph": ["unclassified", "sea_surface", "bathymetry"]
    },
    "cnf": -1,
    "beams": "gt3r",
    "rgt": 202,
    "cycle": 12
}
gdf4 = sliderule.run("atl03x", parms, aoi=aoi)

In [ ]:
gdf4

In [ ]:
plot_atl03(gdf4, 0, 2500)

# (5) Combine ATL03 Filters with ATL24 Classification

In [ ]:
parms = {
    "atl24": {
        "class_ph": ["unclassified", "sea_surface", "bathymetry"]
    },
    "cnf": 2,
    "yapc": {
        "version": 0,
        "score": 100
    },
    "beams": "gt3r",
    "rgt": 202,
    "cycle": 12
}
gdf5 = sliderule.run("atl03x", parms, aoi=aoi)

In [ ]:
gdf5

In [ ]:
plot_atl03(gdf5, 0, 2500)

In [ ]:
plot_atl03(gdf5, 0, 2500, "yapc_score")

# (6) Run ATL06-SR Surface Fitting Algorithm on ATL24 Classified Photons

In [ ]:
parms = {
    "atl24": {
        "class_ph": ["bathymetry"]
    },
    "fit": {
        "res": 10,
        "len": 20,
        "pass_invalid": True
    },
    "cnf": -1,
    "beams": "gt3r",
    "rgt": 202,
    "cycle": 12
}
gdf6 = sliderule.run("atl03x", parms, aoi=aoi)

In [ ]:
gdf6

In [ ]:
plot_atl06(gdf6, 0, 2500)

In [ ]:
plot_atl24(gdf2, 0, 2500)

# (7) Filtered and Ancillary Access to ATL24

In [ ]:
parms = {
    "atl24": {
        "class_ph": ["bathymetry"],
#        "confidence_threshold": 0.6,
#        "invalid_kd": False,
#        "invalid_wind_speed": False,
        "low_confidence": False,
#        "night": True,
#        "sensor_depth_exceeded": False,
        "anc_fields": ["index_ph", "index_seg"]
    },
    "beams": "gt3r",
    "rgt": 202,
    "cycle": 12
}
gdf7 = sliderule.run("atl24x", parms, aoi=aoi)

In [ ]:
gdf7

In [ ]:
plot_atl24(gdf7, 0, 2500)